In [2]:
import os
import re
import random
import math
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [3]:
# パラメータ設定
n = 64
m = 128
LAMBDA = 1
SEED = 5
RATIO = 0.1
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
IMG_NAME = "hadamard"
DIRECTORY = DATA_PATH + "/240717"
SETTING = f"{IMG_NAME}_l1_p-{int(100*RATIO)}_lmd-{int(LAMBDA)}_seed-{SEED}"

In [4]:
def matrix2vector(matrix):
    return matrix.T.flatten()


def vector2matrix(vector, s, t):
    return vector.reshape(s, t).T


def mult_mass(F, h, M):
    N = F.shape[1]
    H = h.reshape(N, M)
    res = F @ H
    return res.flatten()


def images_to_matrix(folder_path, convert_gray=True, rand=True, ratio=RATIO, seed=SEED):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))
    if rand:
        random.seed(seed)
        random.shuffle(files)

    total_files = len(files)
    number_of_files_to_load = int(total_files * ratio)
    selected_files = files[:number_of_files_to_load]
    selected_files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))

    images = []
    use_list = []

    for file in selected_files:
        index = int(re.sub(r"\D", "", file))
        use_list.append(index)
        img = Image.open(os.path.join(folder_path, file))
        if convert_gray:
            img = img.convert("L")
        img_array = np.asarray(img).flatten()
        img_array = img_array / 255
        images.append(img_array)

    return np.column_stack(images), use_list

In [5]:
def fista(F, g, lmd, prox, max_iter=500, tol=1e-2):
    """
    Solve the optimization problem using FISTA:
    min_h ||g - Fh||_2^2 + lambda * ||h||_1

    Parameters:
    - F: numpy array, the matrix F
    - g: numpy array, the vector g
    - lmd: float, the regularization parameter
    - max_iter: int, the maximum number of iterations
    - tol: float, the tolerance for convergence

    Returns:
    - h: numpy array, the solution vector h
    """
    # Initialize variables
    n = F.shape[1]
    t = 1
    h = np.zeros(m * m * n)
    y = np.zeros(m * m * n)

    # Lipschitz constant
    # L = np.linalg.norm(F.T @ F, ord=2) * 3
    L = 4096 * 3
    print(f"L: {L}")
    gamma = 1 / L

    for i in range(max_iter):
        start = time.perf_counter()
        grad = mult_mass(F.T, (mult_mass(F, y, m * m) - g), m * m)
        h_new = prox(y - gamma * grad, gamma * lmd)
        t_new = (1 + np.sqrt(1 + 4 * t**2)) / 2
        y_new = h_new + (t - 1) / t_new * (h_new - h)
        end = time.perf_counter()

        error = np.linalg.norm(y_new - y)
        print(f"iter: {i}, error: {error}, time: {end-start}")
        if error < tol:
            break

        t = t_new.copy()
        h = h_new.copy()
        y = y_new.copy()

    return y_new


def prox_l1(u, tau):
    """Soft thresholding operator for L1 norm."""
    return np.sign(u) * np.maximum(np.abs(u) - tau, 0)

In [6]:
def fista_mat(G, F, lmd, prox, tol=1e-2):
    L = np.linalg.norm(F.T @ F, ord=2) * 3
    gamma = 1 / L
    i = 1
    t = 1
    H_1_prev = np.zeros((G.shape[0], F.shape[0]))
    H_2_prev = np.zeros((G.shape[0], F.shape[0]))

    while True:
        start = time.perf_counter()
        t_new = (1 + np.sqrt(1 + 4 * t**2)) / 2
        grad = -2 * (G - H_2_prev @ F) @ F.T
        H_1 = prox(H_2_prev - (1 / L) * grad, gamma * lmd)
        H_2 = H_1 + (t - 1) / t_new * (H_1 - H_1_prev)
        end = time.perf_counter()

        error = np.linalg.norm(H_2 - H_2_prev, "fro")
        print("iter: {}, error: {}, time: {}".format(i, error, end - start))
        if error < tol:
            break

        t = t_new.copy()
        H_1_prev = H_1.copy()
        H_2_prev = H_2.copy()
        i += 1

    return H_2


def prox_l122(Y, gamma):
    N = Y.shape[1]
    l1_norms = np.sum(np.abs(Y), axis=1)
    factor = (2 * gamma) / (1 + 2 * gamma * N)
    X = np.zeros_like(Y)
    X = np.sign(Y) * np.maximum(np.abs(Y) - factor * l1_norms[:, np.newaxis], 0)
    return X

In [7]:
# load images
G, use = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/")
F, _ = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/")
print("K=", F.shape[1])
white_img = Image.open(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/{IMG_NAME}_1.png").convert("L")
white = np.asarray(white_img).flatten() / 255
white = white[:, np.newaxis]
H1 = np.tile(white, F.shape[1])
F_hat = 2 * F - 1
G_hat = 2 * G - H1

g = matrix2vector(G_hat)

K= 409


In [8]:
h = fista(F_hat.T, g, LAMBDA, prox_l1)

L: 12288
iter: 0, error: 0.6363553728162911, time: 3.946395600003598
iter: 1, error: 0.5711568630352274, time: 3.6282272999960696
iter: 2, error: 0.46694059349454314, time: 3.1618706000008387
iter: 3, error: 0.3694197880141413, time: 2.818320999998832
iter: 4, error: 0.30598931418777237, time: 2.2916430000041146
iter: 5, error: 0.2788087237113029, time: 2.3168503000051714
iter: 6, error: 0.2723939150439043, time: 2.3938363999986905
iter: 7, error: 0.27272265456840233, time: 2.3142083999991883
iter: 8, error: 0.274056737789466, time: 2.3065076999992016
iter: 9, error: 0.2753091744858678, time: 2.307660099999339
iter: 10, error: 0.2765133545913381, time: 2.3461179999940214
iter: 11, error: 0.27774419998478045, time: 2.4068277000042144
iter: 12, error: 0.2789716152867015, time: 2.2047363000019686
iter: 13, error: 0.2801592649980892, time: 2.267164299999422
iter: 14, error: 0.28121413002247586, time: 2.132965400000103
iter: 15, error: 0.28208002342906136, time: 2.1547559999962687
iter: 16,

In [9]:
H = fista_mat(G_hat, F_hat, LAMBDA, prox_l122)

iter: 1, error: 1.4717562029750066, time: 2.979830199998105
iter: 2, error: 0.6539898823485142, time: 3.371729399994365
iter: 3, error: 0.3939620325673145, time: 3.3398687000008067
iter: 4, error: 0.34276367485444265, time: 2.995207300002221
iter: 5, error: 0.32533875646957366, time: 3.0157723000011174
iter: 6, error: 0.3136995777715556, time: 2.9761381999996956
iter: 7, error: 0.30440882614523884, time: 3.170561999999336
iter: 8, error: 0.296823753335836, time: 3.12111410000216
iter: 9, error: 0.2905791392826056, time: 3.0150840999995125
iter: 10, error: 0.2853232242205279, time: 3.063161300000502
iter: 11, error: 0.2808258803259682, time: 2.9656396999998833
iter: 12, error: 0.2768854911606187, time: 2.9541000999961398
iter: 13, error: 0.2732901595164339, time: 2.9422564999986207
iter: 14, error: 0.2698968703306303, time: 2.8817319000008865
iter: 15, error: 0.2666163755697831, time: 2.947001000000455
iter: 16, error: 0.26334814818506447, time: 3.0448045000011916
iter: 17, error: 0.260

In [ ]:
H_mat = H

In [ ]:
if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)
if not os.path.exists(DIRECTORY + "/systemMatrix"):
    os.makedirs(DIRECTORY + "/systemMatrix")

In [ ]:
H = vector2matrix(h, n**2, m**2)
np.save(f"{DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy", H)

SAMPLE_NAME = "Cameraman64"
sample_image = Image.open(f"{DATA_PATH}/sample_image64/{SAMPLE_NAME}.png").convert("L")
sample_image = np.asarray(sample_image).flatten() / 255

Hf = H @ sample_image
Hf_img = Hf.reshape(m, m)
Hf_img = np.clip(Hf_img, 0, 1)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode="L")

FILENAME = f"{SAMPLE_NAME}_{SETTING}.png"
fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_pil, cmap="gray")
ax.axis("off")
fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
plt.show()

In [ ]:
H_true = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy")
rem = np.linalg.norm(H_true - H, "fro")
print(rem)